## Installation

Installing Bio, transformers, genomic-benchmarks, and datasets packages.  The Bio package is from Biopython; transformers package for machine learning (pytorch, tensorflow); genomic-benchmarks and datasets from ML-Bioinfo-CEITEC.


In [ ]:
# already set up on Expanse; toggle for colab

# pip install -qq Bio transformers genomic-benchmarks datasets transformers[torch] pyfaidx

Set path prefix for Expanse

In [1]:
path_prefix = "/expanse/lustre/projects/nji102/sgriesmer/"

Import SNP datasets

In [5]:
import pandas as pd

abs_diff_plus_log_odds_scores_norm = pd.read_csv(path_prefix + "DNABERT_2/output/SNP_PRVCS_0_10492-abs_diff_plus_log_odds_scores_norm_results_by_variant.csv", sep=',')
abs_diff_plus_log_odds_scores_norm.head(), abs_diff_plus_log_odds_scores_norm.shape


(   BroadDnd41CtcfUniPk151-ran  BroadDnd41Ezh239875UniPk151-ran  \
 0                   -0.152371                        -0.516994   
 1                   -0.152467                        -0.525674   
 2                   -0.152467                        -0.527461   
 3                   -0.151815                        -0.512467   
 4                   -0.152354                        -0.490052   
 
    BroadGm12878CtcfUniPk151-ran  BroadGm12878Ezh239875UniPk151-ran  \
 0                     -0.196632                           1.052011   
 1                     -0.196641                           1.103598   
 2                     -0.196646                          -0.592170   
 3                     -0.154866                           0.782636   
 4                     -0.196031                          -0.626693   
 
    BroadH1hescChd1a301218aUniPk151-ran  BroadH1hescCtcfUniPk151-ran  \
 0                            -0.360175                    -0.176740   
 1                      

Choose test set for run

In [6]:
import xgboost

In [7]:
from numpy import asarray
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import pyplot

In [8]:
X=abs_diff_plus_log_odds_scores_norm[0:10492]

In [9]:
X

,BroadDnd41CtcfUniPk151-ran,BroadDnd41Ezh239875UniPk151-ran,BroadGm12878CtcfUniPk151-ran,BroadGm12878Ezh239875UniPk151-ran,BroadH1hescChd1a301218aUniPk151-ran,BroadH1hescCtcfUniPk151-ran,BroadH1hescEzh239875UniPk151-ran,BroadH1hescJarid1aab26049UniPk151-ran,BroadH1hescRbbp5a300109aUniPk151-ran,BroadHelas3CtcfUniPk151-ran,...,UwMcf7CtcfUniPk151-ran.1,UwNb4CtcfUniPk151-ran.1,UwNhdfneoCtcfUniPk151-ran.1,UwNhekCtcfUniPk151-ran.1,UwNhlfCtcfUniPk151-ran.1,UwRptecCtcfUniPk151-ran.1,UwSaecCtcfUniPk151-ran.1,UwSknshraCtcfUniPk151-ran.1,UwWerirb1CtcfUniPk151-ran.1,UwWi38CtcfUniPk151-ran.1
0,-0.152371,-0.516994,-0.196632,1.052011,-0.360175,-0.176740,-0.215733,-0.458238,-0.321760,-0.173018,...,-0.339449,-0.221094,-0.331690,-0.275005,-0.251329,-0.126892,-0.448018,-0.258300,-0.209110,-0.324159
1,-0.152467,-0.525674,-0.196641,1.103598,-0.371735,-0.176740,-0.217395,-0.458167,-0.321896,-0.173017,...,-0.334285,-0.212772,-0.373078,-0.239157,-0.253296,-0.330385,-0.483644,-0.256470,-0.213129,-0.314910
2,-0.152467,-0.527461,-0.196646,-0.592170,-0.372986,-0.176740,-0.218783,-0.458309,-0.321879,-0.173011,...,-0.331167,-0.209084,-0.207284,-0.213020,-0.271579,-0.297286,-0.101763,-0.256126,-0.208747,-0.292523
3,-0.151815,-0.512467,-0.154866,0.782636,-0.372483,-0.175419,-0.218645,-0.450011,-0.301708,-0.172897,...,-0.188366,-0.195130,-0.265205,-0.255205,-0.211320,2.219680,-0.357788,-0.222995,1.752914,-0.148877
4,-0.152354,-0.490052,-0.196031,-0.626693,-0.373202,-0.176734,-0.218509,-0.458340,-0.321368,-0.172503,...,-0.225367,-0.154681,-0.394486,-0.317912,-0.024252,-0.219598,-0.151657,-0.248488,-0.198648,-0.240592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10487,-0.152504,-0.324745,-0.196106,0.224476,-0.125396,-0.176740,-0.218678,0.759228,-0.315989,-0.172958,...,-0.336461,-0.208845,0.176763,-0.248566,-0.271785,-0.343386,-0.315607,-0.249074,-0.206069,-0.258533
10488,-0.152504,-0.530447,-0.196645,-0.689816,-0.373257,-0.176740,-0.218739,-0.458166,-0.321996,-0.173018,...,-0.338310,-0.223931,-0.398954,-0.310995,-0.269957,-0.358221,-0.484077,-0.259590,-0.213987,-0.319960
10489,-0.152503,-0.521134,-0.196577,0.122966,-0.351663,-0.176740,-0.218758,-0.338775,-0.298683,-0.173004,...,-0.294816,-0.161028,-0.009545,-0.264841,-0.271459,-0.337745,-0.355046,-0.256848,-0.210824,-0.294575
10490,-0.152427,-0.520293,-0.196631,-0.684794,-0.372896,-0.176739,-0.218741,-0.456871,-0.307955,-0.173006,...,-0.322732,-0.211425,-0.351270,-0.232562,-0.249021,-0.258437,-0.455136,-0.250392,-0.210892,-0.283941


In [10]:
col_lab = list(range(0,1380))
X.set_axis(col_lab, axis="columns", inplace=True)

In [11]:
X

,0,1,2,3,4,5,6,7,8,9,...,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379
0,-0.152371,-0.516994,-0.196632,1.052011,-0.360175,-0.176740,-0.215733,-0.458238,-0.321760,-0.173018,...,-0.339449,-0.221094,-0.331690,-0.275005,-0.251329,-0.126892,-0.448018,-0.258300,-0.209110,-0.324159
1,-0.152467,-0.525674,-0.196641,1.103598,-0.371735,-0.176740,-0.217395,-0.458167,-0.321896,-0.173017,...,-0.334285,-0.212772,-0.373078,-0.239157,-0.253296,-0.330385,-0.483644,-0.256470,-0.213129,-0.314910
2,-0.152467,-0.527461,-0.196646,-0.592170,-0.372986,-0.176740,-0.218783,-0.458309,-0.321879,-0.173011,...,-0.331167,-0.209084,-0.207284,-0.213020,-0.271579,-0.297286,-0.101763,-0.256126,-0.208747,-0.292523
3,-0.151815,-0.512467,-0.154866,0.782636,-0.372483,-0.175419,-0.218645,-0.450011,-0.301708,-0.172897,...,-0.188366,-0.195130,-0.265205,-0.255205,-0.211320,2.219680,-0.357788,-0.222995,1.752914,-0.148877
4,-0.152354,-0.490052,-0.196031,-0.626693,-0.373202,-0.176734,-0.218509,-0.458340,-0.321368,-0.172503,...,-0.225367,-0.154681,-0.394486,-0.317912,-0.024252,-0.219598,-0.151657,-0.248488,-0.198648,-0.240592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10487,-0.152504,-0.324745,-0.196106,0.224476,-0.125396,-0.176740,-0.218678,0.759228,-0.315989,-0.172958,...,-0.336461,-0.208845,0.176763,-0.248566,-0.271785,-0.343386,-0.315607,-0.249074,-0.206069,-0.258533
10488,-0.152504,-0.530447,-0.196645,-0.689816,-0.373257,-0.176740,-0.218739,-0.458166,-0.321996,-0.173018,...,-0.338310,-0.223931,-0.398954,-0.310995,-0.269957,-0.358221,-0.484077,-0.259590,-0.213987,-0.319960
10489,-0.152503,-0.521134,-0.196577,0.122966,-0.351663,-0.176740,-0.218758,-0.338775,-0.298683,-0.173004,...,-0.294816,-0.161028,-0.009545,-0.264841,-0.271459,-0.337745,-0.355046,-0.256848,-0.210824,-0.294575
10490,-0.152427,-0.520293,-0.196631,-0.684794,-0.372896,-0.176739,-0.218741,-0.456871,-0.307955,-0.173006,...,-0.322732,-0.211425,-0.351270,-0.232562,-0.249021,-0.258437,-0.455136,-0.250392,-0.210892,-0.283941


In [12]:
import pandas as pd

snp_prvcs = pd.read_csv(path_prefix + "DNABERT_2/Datasets/PRVCS/REG_dataset_XY-named.csv", sep=',')


In [13]:
y = snp_prvcs['label'][0:10492]

In [14]:
y

0        1
1        1
2        1
3        1
4        1
        ..
10487    0
10488    0
10489    0
10490    0
10491    0
Name: label, Length: 10492, dtype: int64

In [15]:
y[5240:5250]

5240    1
5241    1
5242    1
5243    1
5244    1
5245    1
5246    1
5247    0
5248    0
5249    0
Name: label, dtype: int64

In [16]:
from sklearn.model_selection import train_test_split
test_size = 0.3
seed = 7
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [17]:
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, roc_auc_score

model1 = XGBClassifier() 
model2 = XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5) 
train_model1 = model1.fit(X_train, y_train) 
train_model2 = model2.fit(X_train, y_train) 
pred1 = train_model1.predict(X_test) 
pred2 = train_model2.predict(X_test) 
print("Accuracy_model1: %.4f" % (accuracy_score(y_test, pred1))) 
print("Accuracy_model2: %.4f" % (accuracy_score(y_test, pred2))) 
print("F1_model1: %.4f" % (f1_score(y_test, pred1)))
print("F1_model2: %.4f" % (f1_score(y_test, pred2)))
print("MCC_model1: %.4f" % (matthews_corrcoef(y_test, pred1)))
print("MCC_model2: %.4f" % (matthews_corrcoef(y_test, pred2)))
print("ROC_AUC_model1: %.4f" % (roc_auc_score(y_test, pred1))) # don't know about call: y_score vs. pred
print("ROC_AUC_model2: %.4f" % (roc_auc_score(y_test, pred2))) # don't know about call: y_score vs. pred


Accuracy_model1: 0.6356
Accuracy_model2: 0.6369
F1_model1: 0.6390
F1_model2: 0.6307
MCC_model1: 0.2712
MCC_model2: 0.2744
ROC_AUC_model1: 0.6356
ROC_AUC_model2: 0.6371


In [18]:
pred2[0:10492].sum()

1508

In [19]:
model3 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=7)

train_model3 = model3.fit(X_train, y_train)
pred3 = train_model3.predict(X_test)
print("Accuracy_model3: %.4f" % (accuracy_score(y_test, pred3)))
print("F1_model3: %.4f" % (f1_score(y_test, pred3)))
print("MCC_model3: %.4f" % (matthews_corrcoef(y_test, pred3)))
print("ROC_AUC_model3: %.4f" % (roc_auc_score(y_test, pred3))) # don't know about call: y_score vs. pred

Accuracy_model3: 0.6487
F1_model3: 0.6462
MCC_model3: 0.2976
ROC_AUC_model3: 0.6488


In [20]:
from sklearn.model_selection import GridSearchCV

param_test = {
 'max_depth':[4],
 'min_child_weight':[2],
 'learning_rate':[0.1],
 'n_estimators':[50],
 'objective':['binary:logistic']
 #'objective':['binary:logistic','binary:hinge','binary:logitraw']
}
    
gsearch = GridSearchCV(estimator=XGBClassifier(gamma=0, 
                                               subsample=0.8, 
                                               colsample_bytree=0.8, 
                                               nthread=4, 
                                               scale_pos_weight=1, 
                                               seed=7), 
                       param_grid=param_test, 
                       scoring='roc_auc',
                       n_jobs=4, 
                       verbose=4,
                       return_train_score=True,
                       cv=5,
                       refit=True)

train_model4 = gsearch.fit(X_train, y_train)
pred4 = train_model4.predict(X_test)
print("Accuracy_model4: %.4f" % (accuracy_score(y_test, pred4)))  
print("F1_model4: %.4f" % (f1_score(y_test, pred4)))  
print("MCC_model4: %.4f" % (matthews_corrcoef(y_test, pred4)))  
print("ROC_AUC_model4: %.4f" % (roc_auc_score(y_test, pred4)))  

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Accuracy_model4: 0.6493
F1_model4: 0.6411
MCC_model4: 0.2995
ROC_AUC_model4: 0.6495


In [21]:
len(pred4)

3148

In [22]:
gsearch.cv_results_['params'][gsearch.best_index_]

{'learning_rate': 0.1,
 'max_depth': 4,
 'min_child_weight': 2,
 'n_estimators': 50,
 'objective': 'binary:logistic'}

In [23]:
model_deepsea1 = XGBClassifier(
 reg_alpha = 20,
 reg_lambda = 2000,
 eta = 0.1,
 n_estimators=10,
 objective= 'binary:logistic',
 seed=7)

train_model_deepsea1 = model_deepsea1.fit(X_train, y_train)
pred_deepsea1 = train_model_deepsea1.predict(X_test)
print("Accuracy_model_deepsea1: %.4f" % (accuracy_score(y_test, pred_deepsea1)))
print("F1_model_deepsea1: %.4f" % (f1_score(y_test, pred_deepsea1)))
print("MCC_model_deepsea1: %.4f" % (matthews_corrcoef(y_test, pred_deepsea1)))
print("ROC_AUC_model_deepsea1: %.4f" % (roc_auc_score(y_test, pred_deepsea1))) # don't know about call: y_score vs. pred

Accuracy_model_deepsea1: 0.6258
F1_model_deepsea1: 0.6110
MCC_model_deepsea1: 0.2532
ROC_AUC_model_deepsea1: 0.6262


In [24]:
model_deepsea2 = XGBClassifier(
 reg_alpha = 0,
 reg_lambda = 10,
 eta = 0.1,
 n_estimators=100,
 objective= 'binary:logistic',
 seed=7)

train_model_deepsea2 = model_deepsea2.fit(X_train, y_train)
pred_deepsea2 = train_model_deepsea2.predict(X_test)
print("Accuracy_model_deepsea1: %.4f" % (accuracy_score(y_test, pred_deepsea2)))
print("F1_model_deepsea2: %.4f" % (f1_score(y_test, pred_deepsea2)))
print("MCC_model_deepsea2: %.4f" % (matthews_corrcoef(y_test, pred_deepsea2)))
print("ROC_AUC_model_deepsea2: %.4f" % (roc_auc_score(y_test, pred_deepsea2))) # don't know about call: y_score vs. pred

Accuracy_model_deepsea1: 0.6407
F1_model_deepsea2: 0.6362
MCC_model_deepsea2: 0.2819
ROC_AUC_model_deepsea2: 0.6409


In [25]:
# Save best model

In [26]:
from joblib import dump, load

best_estimator = train_model4
dump(best_estimator, path_prefix + "DNABERT_2/Output_Models/" + "model_xpg_boost_PRVCS_REG.joblib")

['/expanse/lustre/projects/nji102/sgriesmer/DNABERT_2/Output_Models/model_xpg_boost_PRVCS_REG.joblib']

In [27]:
test_estimator = load('/expanse/lustre/projects/nji102/sgriesmer/DNABERT_2/Output_Models/model_xpg_boost_PRVCS_REG.joblib')

In [28]:
test_pred = test_estimator.predict(X_test)

In [29]:
test_pred.sum()/3148

0.4729987293519695

In [30]:
# Load SNP data

In [31]:
SNPlt01_abs_diff_plus_log_odds_scores_norm = pd.read_csv(path_prefix + "DNABERT_2/output/lt05_igap_ran_0_10000-abs_diff_plus_log_odds_scores_norm_results_by_variant.csv", sep=',')

In [32]:
# Set model names to numbers

In [33]:
SNPlt01_abs_diff_plus_log_odds_scores_norm.set_axis(col_lab, axis="columns", inplace=True)

In [34]:
SNPlt01_abs_diff_plus_log_odds_scores_norm

,0,1,2,3,4,5,6,7,8,9,...,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379
0,-0.134340,-0.176669,-0.167205,1.136512,0.598432,-0.162685,-0.172438,0.035790,-0.143567,-0.154109,...,-0.316542,-0.200087,-0.322255,-0.241135,-0.097094,-0.150122,-0.420258,-0.225923,-0.196953,-0.289094
1,-0.134376,-0.366801,-0.167250,-0.652682,-0.361398,-0.162674,-0.203901,-0.443247,-0.298716,-0.154114,...,0.419565,-0.195666,-0.377331,-0.174015,-0.222661,-0.306294,-0.472069,-0.218917,-0.197477,-0.280357
2,-0.023767,-0.197831,-0.167147,0.070671,-0.200981,0.235543,-0.203821,-0.207416,-0.223960,-0.147906,...,2.071424,-0.186225,-0.038674,0.445475,0.074265,0.001689,-0.210508,-0.193521,-0.186807,0.506437
3,-0.134375,-0.437801,-0.167141,-0.155040,-0.285044,-0.162685,-0.203818,-0.269907,-0.297292,-0.154112,...,-0.319017,-0.198726,-0.384140,-0.275544,-0.037049,-0.276440,-0.447298,-0.224284,-0.196686,-0.247948
4,-0.134354,-0.498539,-0.167261,-0.653013,-0.356434,-0.162685,-0.203906,-0.443459,-0.298768,-0.154109,...,-0.278643,-0.200142,-0.378112,-0.200160,-0.197726,-0.271060,-0.457370,-0.226998,-0.196294,-0.271508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.134333,-0.102992,-0.167079,0.438134,-0.358858,-0.162685,-0.203737,5.375807,-0.297380,-0.154109,...,-0.316421,-0.195129,-0.325719,-0.169471,-0.159812,0.021548,-0.315698,-0.225919,-0.197648,-0.276390
9996,-0.130458,-0.029596,-0.127534,1.780980,-0.002129,-0.162602,-0.203783,-0.264749,0.331581,-0.145322,...,0.118872,-0.152362,-0.317824,0.050219,0.247068,-0.077249,-0.193526,-0.225535,-0.169228,0.006952
9997,-0.134323,-0.276844,-0.167105,-0.526303,-0.351880,-0.162685,-0.198805,-0.370694,-0.266654,-0.154111,...,-0.309555,-0.190732,-0.338031,-0.288699,-0.218013,-0.282265,-0.440474,-0.219109,-0.186410,-0.270539
9998,-0.134193,0.185418,-0.165822,-0.172234,-0.355526,-0.162660,-0.203906,-0.435158,-0.297253,-0.153950,...,-0.224890,-0.168852,-0.143209,-0.260668,-0.245686,-0.070232,-0.192810,-0.107638,-0.195652,0.022638


In [35]:
SNPlt01_pred = test_estimator.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm)

In [36]:
SNPlt01_pred.sum()

9999

In [37]:
SNPgt5_abs_diff_plus_log_odds_scores_norm = pd.read_csv(path_prefix + "DNABERT_2/output/gt5_igap_ran_0_10000-abs_diff_plus_log_odds_scores_norm_results_by_variant.csv", sep=',')

In [38]:
SNPgt5_abs_diff_plus_log_odds_scores_norm.set_axis(col_lab, axis="columns", inplace=True)

In [39]:
SNPgt5_abs_diff_plus_log_odds_scores_norm

,0,1,2,3,4,5,6,7,8,9,...,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379
0,-0.134340,-0.176669,-0.167205,1.136512,0.598432,-0.162685,-0.172438,0.035790,-0.143567,-0.154109,...,-0.316542,-0.200087,-0.322255,-0.241135,-0.097094,-0.150122,-0.420258,-0.225923,-0.196953,-0.289094
1,-0.134376,-0.366801,-0.167250,-0.652682,-0.361398,-0.162674,-0.203901,-0.443247,-0.298716,-0.154114,...,0.419565,-0.195666,-0.377331,-0.174015,-0.222661,-0.306294,-0.472069,-0.218917,-0.197477,-0.280357
2,-0.023767,-0.197831,-0.167147,0.070671,-0.200981,0.235543,-0.203821,-0.207416,-0.223960,-0.147906,...,2.071424,-0.186225,-0.038674,0.445475,0.074265,0.001689,-0.210508,-0.193521,-0.186807,0.506437
3,-0.134375,-0.437801,-0.167141,-0.155040,-0.285044,-0.162685,-0.203818,-0.269907,-0.297292,-0.154112,...,-0.319017,-0.198726,-0.384140,-0.275544,-0.037049,-0.276440,-0.447298,-0.224284,-0.196686,-0.247948
4,-0.134354,-0.498539,-0.167261,-0.653013,-0.356434,-0.162685,-0.203906,-0.443459,-0.298768,-0.154109,...,-0.278643,-0.200142,-0.378112,-0.200160,-0.197726,-0.271060,-0.457370,-0.226998,-0.196294,-0.271508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.134333,-0.102992,-0.167079,0.438134,-0.358858,-0.162685,-0.203737,5.375807,-0.297380,-0.154109,...,-0.316421,-0.195129,-0.325719,-0.169471,-0.159812,0.021548,-0.315698,-0.225919,-0.197648,-0.276390
9996,-0.130458,-0.029596,-0.127534,1.780980,-0.002129,-0.162602,-0.203783,-0.264749,0.331581,-0.145322,...,0.118872,-0.152362,-0.317824,0.050219,0.247068,-0.077249,-0.193526,-0.225535,-0.169228,0.006952
9997,-0.134323,-0.276844,-0.167105,-0.526303,-0.351880,-0.162685,-0.198805,-0.370694,-0.266654,-0.154111,...,-0.309555,-0.190732,-0.338031,-0.288699,-0.218013,-0.282265,-0.440474,-0.219109,-0.186410,-0.270539
9998,-0.134193,0.185418,-0.165822,-0.172234,-0.355526,-0.162660,-0.203906,-0.435158,-0.297253,-0.153950,...,-0.224890,-0.168852,-0.143209,-0.260668,-0.245686,-0.070232,-0.192810,-0.107638,-0.195652,0.022638


In [40]:
SNPgt5_pred = test_estimator.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm)

In [41]:
SNPgt5_pred.sum()

9999

In [42]:
# Try other models

In [43]:
pred1_lt01 = train_model1.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm) 
pred2_lt01 = train_model2.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm) 
pred3_lt01 = train_model3.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm)
pred4_lt01 = train_model4.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm)
pred_deepsea1_lt01 = train_model_deepsea1.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm)
pred_deepsea2_lt01 = train_model_deepsea2.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm)

In [44]:
for i in [
    pred1_lt01,
    pred2_lt01,
    pred3_lt01,
    pred4_lt01,
    pred_deepsea1_lt01,
    pred_deepsea2_lt01,
  ]:
    print(i.sum())

9925
9915
9973
9999
10000
9991


In [45]:
pred1_gt5 = train_model1.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm) 
pred2_gt5 = train_model2.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm) 
pred3_gt5 = train_model3.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm)
pred4_gt5 = train_model4.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm)
pred_deepsea1_gt5 = train_model_deepsea1.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm)
pred_deepsea2_gt5 = train_model_deepsea2.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm)

In [46]:
for i in [
    pred1_gt5,
    pred2_gt5,
    pred3_gt5,
    pred4_gt5,
    pred_deepsea1_gt5,
    pred_deepsea2_gt5,
  ]:
    print(i.sum())

9925
9915
9973
9999
10000
9991
